In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder,LabelEncoder
import pickle

In [4]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'],axis=1)

In [6]:
labelencoder=LabelEncoder()
data['Gender']=labelencoder.fit_transform(data['Gender'])

In [7]:
ohe_encoder=OneHotEncoder(handle_unknown='ignore')
ohe_data=ohe_encoder.fit_transform(data[['Geography']]).toarray()
ohe_df=pd.DataFrame(ohe_data, columns=ohe_encoder.get_feature_names_out(['Geography']))
ohe_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [8]:
data=pd.concat([data.drop('Geography',axis=1),ohe_df],axis=1)

In [9]:
X = data.drop('EstimatedSalary',axis=1)
y= data['EstimatedSalary']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
with open('Rscaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('Rlabelencoder.pkl', 'wb') as f:
    pickle.dump(scaler, f)

with open('Rohe.pkl', 'wb') as f:
    pickle.dump(scaler, f)

## ANN for Regression

In [11]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [12]:
model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae'])

model.summary()

d:\GenAI\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [13]:
import datetime
log_dir = "logs/regfit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [14]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [15]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[tensorboard_callback, early_stopping])

Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 13410961408.0000 - mae: 100453.5391 - val_loss: 13696493568.0000 - val_mae: 102031.4062
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 13456556032.0000 - mae: 100816.2812 - val_loss: 13543821312.0000 - val_mae: 101284.4766
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 13161088000.0000 - mae: 99428.0469 - val_loss: 13131991040.0000 - val_mae: 99272.9531
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 12473894912.0000 - mae: 95742.9062 - val_loss: 12384718848.0000 - val_mae: 95612.1094
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 11982724096.0000 - mae: 93308.2188 - val_loss: 11282298880.0000 - val_mae: 90178.9766
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 10645489664.0000 - mae: 87135.3828 - val_loss: 9888646144.0000 - val_mae: 83269.9062
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 9165908992.0000 - mae: 79228.0781 - val_loss: 8356995

In [16]:
model.save('Regressionmodel.h5')

In [17]:
# load Tensorboard Extension
%load_ext tensorboard

In [18]:
%tensorboard --logdir logs/regfit20250329-215346

In [19]:
#Evaluation on test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")

63/63 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 3411939072.0000 - mae: 50731.8594
Test Loss: 3358754304.0
